In [106]:
import yaml
from pprint import pprint
import dicttoxml
# there is also dict2xml available in pypy (python3)
import xml.etree.ElementTree as ET


In [107]:
with open('../data/zynqmon_2.yml') as f:
    y = yaml.load(f, Loader=yaml.FullLoader)
    pprint(y)


{'cm_hw_rev': '2B',
 'config': [{'count': 20,
             'extra': 'Table=CM_FF_MON;Row=_3;Column=Temp_C;Status=2;',
             'mcu_call': 'firefly_temps',
             'name': 'firefly',
             'names': ['F1_1  12 Tx',
                       'F1_1  12 Rx',
                       'F1_2  12 Tx',
                       'F1_2  12 Rx',
                       'F1_3  12 Tx',
                       'F1_3  12 Rx',
                       'F1_4 4 XCVR',
                       'F1_5 4 XCVR',
                       'F1_6 4 XCVR',
                       'F1_7 4 XCVR',
                       'F2_1  12 Tx',
                       'F2_1  12 Rx',
                       'F2_2  12 Tx',
                       'F2_2  12 Rx',
                       'F2_3  12 Tx',
                       'F2_3  12 Rx',
                       'F2_4 4 XCVR',
                       'F2_5 4 XCVR',
                       'F2_6 4 XCVR',
                       'F2_7 4 XCVR'],
             'start': 0,
             'type': '

In [108]:
# This is the parent (root) tag
# onto which other tags would be
# created
data = ET.Element('node')
data.set('id', 'PL_MEM')


In [109]:
cm = ET.SubElement(data, 'node')
cm.set('id', 'CM')
cm.set('address', '0x00000000')


In [110]:
config = y['config']

for c in config: # loop over entries in configuration (sensor category)
    print("here", c['name'])
    if not 'start' in c:
        pprint(c)
    start = c['start']
    count = c['count']
    i = 0 # counter over the number of sensors within a category
    names = c['names']
    if 'prefixes' in c:
        prefixes = c['prefixes']
    else:
        prefixes = ' '
    for p in prefixes:
        for n in names: # loop over sensor names
            node = ET.SubElement(cm, 'node')
            if ( len(p.lstrip()) > 0 ):
                id = p +"_" +  n.replace(' ', '_')
            else:
                id = n.replace(' ', '_')
            node.set('id', id)
            addr = int((start + i)/2) ## address is half of the sensor address since these are 32 bit addresses
            remain = (start + i) % 2
            node.set('address', str(hex(addr)))
            i += 1 # offset for next sensor
            # this appears to be on all the nodes
            node.set("permission", "r")
            # set parameter based on the type
            if ( c['type'] == 'int8' ):
                width=8
                format = "d"
            elif ( c['type'] == 'fp16' ):
                width=16
                format = "fp16"
            elif ( c['type'] == 'char' ):
                width=8
                format = "c"
            elif ( c['type'] == 'uint32_t' ):
                width=32
                format = "u"
            else:
                print("ERROR: unknown type", c['type'])
                break
            mask = (1 << width) - 1
            if (remain == 0):
                node.set('mask', "0x{0:08X}".format(mask))
            else:
                node.set('mask', "0x{0:08X}".format(mask<<16))
            if 'extra' in c:
                extra = c['extra']
            else:
                extra = ""
            node.set('parameters', "format=" + format + ";" + extra)
        if 'size' in c:
            print("size:", c['size'], c['name'])
            node.set('size', str(hex(c['size'])))


here firefly
here psmon
here adcmon
here uptime
here gitversion
size: 5 gitversion
here fpga


In [111]:
s_xml = ET.tostring(data)


In [112]:
from bs4 import BeautifulSoup

bs = BeautifulSoup(s_xml, 'xml')
pretty_xml = bs.prettify()
print(pretty_xml)


<?xml version="1.0" encoding="utf-8"?>
<node id="PL_MEM">
 <node address="0x00000000" id="CM">
  <node address="0x0" id="F1_1__12_Tx" mask="0x000000FF" parameters="format=d;Table=CM_FF_MON;Row=_3;Column=Temp_C;Status=2;" permission="r"/>
  <node address="0x0" id="F1_1__12_Rx" mask="0x00FF0000" parameters="format=d;Table=CM_FF_MON;Row=_3;Column=Temp_C;Status=2;" permission="r"/>
  <node address="0x1" id="F1_2__12_Tx" mask="0x000000FF" parameters="format=d;Table=CM_FF_MON;Row=_3;Column=Temp_C;Status=2;" permission="r"/>
  <node address="0x1" id="F1_2__12_Rx" mask="0x00FF0000" parameters="format=d;Table=CM_FF_MON;Row=_3;Column=Temp_C;Status=2;" permission="r"/>
  <node address="0x2" id="F1_3__12_Tx" mask="0x000000FF" parameters="format=d;Table=CM_FF_MON;Row=_3;Column=Temp_C;Status=2;" permission="r"/>
  <node address="0x2" id="F1_3__12_Rx" mask="0x00FF0000" parameters="format=d;Table=CM_FF_MON;Row=_3;Column=Temp_C;Status=2;" permission="r"/>
  <node address="0x3" id="F1_4_4_XCVR" mask="0x

In [113]:
def calcSize(c: dict) -> int:
    if 'size' in c:
        size = c['size']*2 # extra factor of 2 for 16 to 32 bit
    else:
        size = 1
    if '32' in c['type']:
        size = size * 2
    return size


In [114]:
# check overlaps
# create an object with a name, and a start end end register
# do so for every entry
# then ensure that no two objects overlap

# create a class to hold a name, start, end, and size
# add a method to check if two objects overlap
# and a pretty print method
from itertools import chain
class reg:
    def __init__(self, name, start, end, size):
        self.name = name
        self.start = start
        self.end = end
        self.size = size
    def __str__(self):
        return "name: " + self.name + " start: " + str(self.start) + " end: " + str(self.end) + " size: " + str(self.size)
    def __repr__(self):
        return "name: " + self.name + " start: " + str(self.start) + " end: " + str(self.end) + " size: " + str(self.size)
    def overlaps(self, other):
        if (self.start <= other.start and self.end >= other.start):
            return True
        if (self.start <= other.end and self.end >= other.end):
            return True
        if (self.start >= other.start and self.end <= other.end):
            return True
        return False

# create a list of objects
entries = []
for c in config:  # loop over entries in configuration (sensor category)
    if not 'start' in c:
        pprint(c)
    start = c['start']
    count = c['count']
    i = 0  # counter over the number of sensors within a category
    names = c['names']
    if 'prefixes' in c:
        prefixes = c['prefixes']
    else:
        prefixes = ' '
    size = calcSize(c)    
    thislength = len(prefixes) * len(names)*size
    entries.append(reg(c['name'], start, start + thislength - 1, thislength))
pprint(entries)

# check for overlaps
for i in range(len(entries)):
    for j in range(i+1, len(entries)):
        if entries[i].overlaps(entries[j]):
            print(f"{entries[i].name} overlaps with {entries[j].name}")



[name: firefly start: 0 end: 19 size: 20,
 name: psmon start: 32 end: 111 size: 80,
 name: adcmon start: 128 end: 148 size: 21,
 name: uptime start: 192 end: 193 size: 2,
 name: gitversion start: 118 end: 127 size: 10,
 name: fpga start: 150 end: 157 size: 8]
